In [1]:
import sys
from scipy.spatial import distance
import numpy as np
import pandas as pd
import random
from copy import deepcopy

In [2]:
#df_data = pd.read_csv(sys.stdin, sep=',', header=None)
df_data = pd.read_csv('dataset1.dat', sep=',', header=None)
data_np = np.array(df_data)
canopy_data = df_data

In [3]:
# Empty lists for canopy function

random_centroids_list = []
total_distance = []
sum_distance = []
centroids = []


def canopy(data, k):
    if k == 1:
        # Random centroid taken from dataset
        random_point = (canopy_data.sample(1))
        centroids.append(random_point)
        # print(centroids)
        return centroids

    else:
        # Random centroid taken from dataset
        random_point = (canopy_data.sample(1))
        centroids.append(random_point)
        data = canopy_data.drop(random_point.index, axis=0, inplace=True)

        for i in range(k - 1):
            c_distance = distance.cdist(canopy_data, random_point, 'euclidean')
            sum_distance.append(c_distance)
            if k > 1:
                total_distance = np.add(sum_distance, c_distance)
                sum_distance.clear()
            else:
                pass

            index = np.argmax(c_distance)
            # print(index)
            new_centroid = canopy_data.iloc[[index]]
            centroids.append(new_centroid)
            data = canopy_data.drop(new_centroid.index, axis=0, inplace=True)

        # convert to numpy array
        centroid_array = np.asarray(centroids)
        centroids.clear()
        return centroid_array

In [4]:
# Rows and Colunms of particular data
n = df_data.shape[0]
# Number of features in the data
c = df_data.shape[1]

# Cluster Assignment function of Kmeans
def assign_cluster(k, dataset, centers_new):
    dist = {}
    # print("kkk",k)
    clusters = np.zeros(n)
    distances = np.zeros((n, k))
    # print(distances.shape)
    for i in range(k):
        dist[i] = []
        distances[:, i] = np.linalg.norm(dataset - centers_new[i], axis=1)
    # print(distances)
    clusters = np.argmin(distances, axis=1)
    # print(clusters)
    # centers_old = deepcopy(centers_new)
    # print("old",centers_old)
    # print(clusters)

    # Assign cluster points to each centroids and store it as a dictonary
    for c in range(len(clusters)):
        # print(range(len(clusters)))
        if clusters[c] not in dist:
            dist[clusters[c]] = [dataset[c]]
            # print(clusters[c])
        else:
            dist[clusters[c]].append(dataset[c])
    # print("dist",clusters)
    return dist

In [5]:
# After assign cluster function, new centroids gets updated
def update_cluster(dist, centers_new):
    for k1, v1 in dist.items():
        # print(k1,v1)
        centers_new[k1] = np.mean(v1, axis=0)
    # print("update",centers_new)
    return centers_new

In [6]:
# silhouettes method calculates silhouettes score and returns mean si.
def silhouettes(k_Means_output):
    a_i = []
    b_i = []
    list_bi_mean = []
    si = []
    # sc = []
    # a
    for k, v in k_Means_output.items():

        b = list(k_Means_output[k])
        # print(b)
        for p1 in b:
            intra_distance = 0
            for p2 in b:
                if tuple(p1) != tuple(p2):
                    intra_distance += distance.euclidean(tuple(p1), tuple(p2))
                else:
                    continue
            if len(b) == 1:
                a_i_mean = intra_distance
            else:
                a_i_mean = intra_distance / (len(k_Means_output[k]) - 1)
            a_i.append(a_i_mean)

            for key, val in k_Means_output.items():
                if key != k:
                    inter_cluster_distance = 0

                    c = list(k_Means_output[key])
                    for p3 in c:
                        inter_cluster_distance += distance.euclidean(tuple(p1), tuple(p3))
                else:
                    continue
                b_i_mean = inter_cluster_distance / len(k_Means_output[key])
                list_bi_mean.append(b_i_mean)
            b_i.append(min(list_bi_mean))
            # print(list_bi_mean)

            # for i in range(len(a_i)):
            s_i = (min(list_bi_mean) - a_i_mean) / max(min(list_bi_mean), a_i_mean)
            si.append(s_i)
    mean_si = np.mean(si)
    # sc.append(mean_si)
    return mean_si

In [7]:
# K-means function calculates and recalculates centroids and until centroids #converge
def k_means(dataset, previous_centroids, k):
    # dist = {}
    count = 0
    # print(len()previous_centroids)
    centers_old = np.zeros((len(previous_centroids), c))  # to store old centers
    centers_new = deepcopy(previous_centroids)  # Store new centers

    dataset.shape
    # clusters = np.zeros(n)
    # distances = np.zeros((n,k))
    # print(centers_new.shape,centers_old.shape)
    error = np.linalg.norm(centers_new - centers_old)
    while error != 0:
        # print('yes')
        count += 1
        # print(centers_new)
        dist = assign_cluster(k, dataset, centers_new)
        # Old centeroids are centroids of previous iteration
        # new centroids are newly genrated centroids for next iteration
        centers_old = deepcopy(centers_new)
        centers_new = update_cluster(dist, centers_new)
        # print(centers_new,centers_old)
        error = np.linalg.norm(centers_new - centers_old)
        # print("error",error)
    dist = assign_cluster(k, dataset, centers_new)
    # centers_old = np.zeros(len(previous_centroids))

    return dist, centers_new

In [8]:
# To handle an anomalies in data such as nan, string values.
# It exists if values are empty or non real 
if df_data.isnull().values.any() or ~df_data.applymap(np.real).all().all():
    exit()
else:
    # Else entire k-means algorithm runs

    k = [2,3,4,5,6]
    sc = []
    for i in k:
        #  canopy_centroids = np.zeros((i,c))
        #  canopy_centroids = canopy(canopy_data, i)
        centroid_store = []
        centroid_store = data_np[np.random.randint(0, n - 1, size=i)]
        final_kmeans, new = k_means(data_np, centroid_store, i)
        sil_score = silhouettes(final_kmeans)
        # print(i,sil_score)
        sc.append(sil_score)
    # print(sc)
    # for k1, v1 in k_Means_output.items():
    #     print(k1, len(v1))

    max_sc = max(sc)
    # print(max_sc)
    optimum_k = k[sc.index(max_sc)]
    # print(optimum_k)
    # final_kmeans, final_centroids = k_means(data_np, canopy_centroids, optimum_k)

    # Anomaly detection
    threshold = (0.01 * len(df_data))

    # 1) the small clusters with less than a threshold (e.g., 1% of total number of data points)
    # 2) isolation data points not belong to any cluster

    arr = []
    outlier = []
    for k, v in final_kmeans.items():
        arr.append([k, (np.mean(v, axis=0) + 2 * np.std(v, axis=0)), (np.mean(v, axis=0) - 2 * np.std(v, axis=0))])

    clust_dict = dict(sorted(final_kmeans.items()))
    clustr_count = {}
    for k, v in clust_dict.items():
        clustr_count[k] = len(v)
    # print(clustr_count)
    # Length od cluster is 1, consider them as isolated points and append to #outliers.

    for key, value in clustr_count.items():
        if value == 1:
            outlier.append(value)

        elif value < threshold:
            for i in clust_dict[key]:
                outlier.append(i)
    
    # a data point belongs to a cluster with more than 2 standard deviations #(i.e., 95% confidence)

    for i, j in zip(clust_dict.keys(), sorted(arr)):
        for y in final_kmeans[i]:
            if (np.greater(y, arr[i][1]).all()) and ~(np.less(y, arr[i][2]).any()):
                outlier.append(y)
    
    # Final outliers of a dataset
    for outlr in outlier:
        print(outlr)

[ -771.2 -1582.8]
[-1582.  -1582.7]
